MACHINE LEARNING

Mi objetivo es encontrar peliculas similares y para encontrar esa similitud me basaria en las puntuaciones y generos.

In [93]:
import pandas as pd  
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import TfidfVectorizer 

In [98]:
df = pd.read_csv("plataformas.csv", sep=",")

In [99]:
df.head(2)

,Unnamed: 0,id,title,director,cast,country,date_added,release_year,rating,listed_in,description,type,duration_int,duration_type,scored
0,0,as1,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,movie,113,min,3.5
1,1,as2,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,movie,110,min,3.5


Voy a comenzar a eliminar las columnas que no necesito. En mi caso voy a trabajar con la columna id,title,director,rating,type,listed_in y scored

In [100]:
df_entrenamiento = df.drop(columns = ["Unnamed: 0", "cast", "country", "date_added", "release_year", "duration_int", "duration_type","director", "description","id","rating","type"])

In [101]:
df_entrenamiento

,title,listed_in,scored
0,the grand seduction,"comedy, drama",3.5
1,take care good night,"drama, international",3.5
2,secrets of deception,"action, drama, suspense",3.5
3,pink: staying true,documentary,3.5
4,monster maker,"drama, fantasy",3.5
...,...,...,...
17259,pan's labyrinth,"dramas, international movies, sci-fi & fantasy",3.5
17260,pokémon the movie: power of us,"anime features, children & family movies",3.5
17261,posesif,"dramas, international movies, romantic movies",3.5
17262,reply 1988,"international tv shows, korean tv shows, roman...",3.5


Me fijo si hay datos nulos

In [73]:
df_entrenamiento.isnull().sum()

id           0
title        0
rating       0
listed_in    0
type         0
scored       0
dtype: int64

Obtengo los valores unicos de las diferentes columnas

In [74]:
df_entrenamiento["id"].value_counts()  
df_entrenamiento["title"].value_counts() 
df_entrenamiento["rating"].value_counts() #En este caso veo que hay valores que no corresponden a raiting  
df_entrenamiento["type"].value_counts() 
df_entrenamiento["scored"].value_counts() 
df_entrenamiento["listed_in"].value_counts()


drama                                      1061
comedy                                      608
comedy, drama                               421
drama, suspense                             399
documentary                                 375
                                           ... 
animals & nature, coming of age, family       1
family, fantasy                               1
buddy, family, sports                         1
animation, buddy, family                      1
anime series, teen tv shows                   1
Name: listed_in, Length: 1536, dtype: int64

Veo todos los datos diferentes que tengo

In [75]:
df_entrenamiento["type"].unique() #Tiene solamente 2 valores y son los correctos 
df_entrenamiento["listed_in"].unique()

array(['comedy, drama', 'drama, international', 'action, drama, suspense',
       ..., 'docuseries, reality tv, teen tv shows',
       'crime tv shows, international tv shows, reality tv',
       'anime series, teen tv shows'], dtype=object)

In [14]:
df_entrenamiento["rating"].unique()

array(['g', '13+', 'all', '18+', 'r', 'tv-y', 'tv-y7', 'nr', '16+',
       'tv-pg', '7+', 'tv-14', 'tv-nr', 'tv-g', 'pg-13', 'tv-ma', 'pg',
       'nc-17', 'unrated', '16', 'ages_16_', 'ages_18_', 'all_ages',
       'not_rate', 'tv-y7-fv', 'not rated', '2 seasons', '93 min',
       '4 seasons', '136 min', '91 min', '85 min', '98 min', '89 min',
       '94 min', '86 min', '3 seasons', '121 min', '88 min', '101 min',
       '1 season', '83 min', '100 min', '95 min', '92 min', '96 min',
       '109 min', '99 min', '75 min', '87 min', '67 min', '104 min',
       '107 min', '84 min', '103 min', '105 min', '119 min', '114 min',
       '82 min', '90 min', '130 min', '110 min', '80 min', '6 seasons',
       '97 min', '111 min', '81 min', '49 min', '45 min', '41 min',
       '73 min', '40 min', '36 min', '39 min', '34 min', '47 min',
       '65 min', '37 min', '78 min', '102 min', '129 min', '115 min',
       '112 min', '61 min', '106 min', '76 min', '77 min', '79 min',
       '157 min', '28 mi

In [15]:
df_entrenamiento["scored"].unique() #Scored tiene los valores correctos, los que pertenecen a score

array([3.5, 3.6, 3.4, 3.7, 3.3])

Mi conclucion es que hay datos que no necesito en esta columna. Ej: Min,season,notrate,ect.

Ahora comienzo a eliminarlos y rellenar los datos que van a quedar vacios con not_rated.

In [76]:
clas = ["g","13+","18+""r","tv-y","tv-y7-fv","16+", "tv-pg", "7+", "tv-14", "tv-ma", "pg", "nc-17", "unrated", "pg-13"]  
df_entrenamiento = df_entrenamiento[df_entrenamiento["rating"].isin(clas)]


Relleno

In [78]:
df_entrenamiento["rating"].fillna("not rated", inplace=True) 
df_entrenamiento["listed_in"].fillna("not rated", inplace=True)

C:\Users\Bounset\AppData\Local\Temp\ipykernel_9644\1779072622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entrenamiento["rating"].fillna("not rated", inplace=True)
C:\Users\Bounset\AppData\Local\Temp\ipykernel_9644\1779072622.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entrenamiento["listed_in"].fillna("not rated", inplace=True)


Saco los espacios 

In [79]:
df_entrenamiento['title'] = df_entrenamiento['title'].str.replace(' ', '_')

C:\Users\Bounset\AppData\Local\Temp\ipykernel_9644\792193736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entrenamiento['title'] = df_entrenamiento['title'].str.replace(' ', '_')


In [94]:
df_entrenamiento

,id,title,rating,listed_in,type,scored
0,as1,the_grand_seduction,g,"comedy, drama",movie,3.5
1,as2,take_care_good_night,13+,"drama, international",movie,3.5
2,as3,secrets_of_deception,g,"action, drama, suspense",movie,3.5
3,as4,pink:_staying_true,g,documentary,movie,3.5
4,as5,monster_maker,g,"drama, fantasy",movie,3.5
...,...,...,...,...,...,...
17256,ns3066,messiah,tv-ma,"tv dramas, tv thrillers",tv show,3.6
17258,ns3068,"nisman:_the_prosecutor,_the_president,_and_the...",tv-ma,"crime tv shows, docuseries, international tv s...",tv show,3.6
17261,ns3071,posesif,tv-ma,"dramas, international movies, romantic movies",movie,3.5
17262,ns3072,reply_1988,tv-14,"international tv shows, korean tv shows, roman...",tv show,3.5


In [60]:
df_entrenamiento["title"].values

array(['the_grand_seduction', 'take_care_good_night',
       'secrets_of_deception', ..., 'posesif', 'reply_1988', 'riot'],
      dtype=object)

In [104]:
df_entrenamiento

,title,listed_in,scored
0,the grand seduction,"comedy, drama",3.5
1,take care good night,"drama, international",3.5
2,secrets of deception,"action, drama, suspense",3.5
3,pink: staying true,documentary,3.5
4,monster maker,"drama, fantasy",3.5
...,...,...,...
17259,pan's labyrinth,"dramas, international movies, sci-fi & fantasy",3.5
17260,pokémon the movie: power of us,"anime features, children & family movies",3.5
17261,posesif,"dramas, international movies, romantic movies",3.5
17262,reply 1988,"international tv shows, korean tv shows, roman...",3.5


MODELO MACHINE LEARNING

In [111]:
def get_recommendation(title: str): 
    #Obtengo la fila correspondiente a la pelicula de interes 
    peliculas_idx = df_entrenamiento[df_entrenamiento["title"] == title].index[0]  
    #Creo una instancia de TfidfVectorizer y creo la matriz de frecuencia de términos  
    tfidf = TfidfVectorizer(stop_words="english") 
    tfidf_matrix = tfidf.fit_transform(df_entrenamiento["listed_in"])  
    #Calculo la similitud del coseno 
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix) 
    #Obtengo las peliculas por similitud 
    peliculas_similares = list(enumerate(cosine_sim[peliculas_idx]))  
    #Ordeno las peliculas por similitud 
    agregar_peliculas = sorted(peliculas_similares, key=lambda x: x[1], reverse=True)
    #Obtengo los indices de las 5 peliculas mas similares  
    top_similar = [i[0] for i in agregar_peliculas[1:6]] 
    #Obtengo los nombres de las pelicualas mas similares 
    top_similares_titulos = df_entrenamiento["title"].iloc[top_similar] 
    #Obtengo los scores de las peliculas mas similares 
    top_similares_score = [agregar_peliculas[i][1] for i in top_similar] 
    #Obtengo las peliculas similares y a la ves que presentan mayor score  
    #Zip toma dos iterables y los combia en una lista de tuplas en este caso los 2 escores 
    #Desempaqueto las tuplas en la lista ordenada, donde _ representa la variable score y x el titulo.
    peliculas_recomendadas = [x for _, x in sorted(zip(top_similares_score, top_similares_titulos), reverse=True)][:5]
    return {"titulo": peliculas_recomendadas}


In [113]:
get_recommendation("the grand seduction")

{'titulo': ['z: the beginning of everything',
  'you, me and him',
  'walter',
  'waiting on mary',
  "summer '03"]}

Aca creo un nuevo csv que va a ser para mi funcion de machine learning, para llamarlo luego en mi api

In [109]:
df_entrenamiento.to_csv("ML.csv", index =False)